In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
import math
import statistics
import re
import json

In [2]:
business=pd.read_csv("yelp_business.csv")

In [3]:
business= business[business.state == "OH"]
business= business[business.is_open == 1]


In [4]:
def category_splitter(category_string):
    categories = category_string.split(";")   
    return categories
business['categories'] = business['categories'].map(lambda x: category_splitter(x))


In [5]:
def first_list(category_list):
    category_list = category_list[0]
    return category_list
business['categories'] = business['categories'].map(lambda x: first_list(x).lower())
business['categories'] = business['categories'].map(lambda x: re.sub(r'\'', '', x).lower())

business.drop(["business_id","neighborhood","address","review_count","is_open"], axis=1)


,name,city,state,postal_code,latitude,longitude,stars,categories
4,"""Brick House Tavern + Tap""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,american (new)
28,"""Panera Bread""",Elyria,OH,44035,41.343078,-82.067140,2.0,soup
35,"""Brewster's Pub""",Munroe Falls,OH,44308,41.136622,-81.439259,4.0,nightlife
40,"""Yuzu""",Lakewood,OH,44107,41.476846,-81.786971,3.5,nightlife
62,"""Falls Auto Body""",Cuyahoga Falls,OH,44223,41.180485,-81.509452,5.0,automotive
87,"""Havel's Flowers & Greenhouses""",Mentor,OH,44060,41.677768,-81.306864,3.5,florists
89,"""International Exposition Center""",Cleveland,OH,44135,41.398505,-81.853220,4.0,venues & event spaces
94,"""Mitchell's Ice Cream - Solon Shop""",Solon,OH,44139,41.392049,-81.442635,4.5,ice cream & frozen yogurt
117,"""Barrio""",Cleveland Heights,OH,44106,41.500586,-81.592278,3.5,food
144,"""Exotic Nails Touch""",North Olmsted,OH,44070,41.417527,-81.922710,4.0,beauty & spas


In [6]:
business.to_csv("oh_data.csv")

In [8]:
# for typeahead words
raw_unique_cats = list(np.unique(business.categories))

unique_cats = [re.sub(r'\'', '', i).lower() for i in raw_unique_cats]

print(unique_cats)

['accessories', 'accountants', 'active life', 'acupuncture', 'adult', 'adult education', 'adult entertainment', 'african', 'air duct cleaning', 'airlines', 'airport shuttles', 'airports', 'american (new)', 'american (traditional)', 'amusement parks', 'animal shelters', 'antiques', 'apartments', 'appliances', 'appliances & repair', 'arcades', 'archery', 'art classes', 'art schools', 'art supplies', 'arts & crafts', 'arts & entertainment', 'asian fusion', 'auction houses', 'auto customization', 'auto detailing', 'auto glass services', 'auto loan providers', 'auto parts & supplies', 'auto repair', 'auto upholstery', 'automotive', 'baby gear & furniture', 'bagels', 'bakeries', 'banks & credit unions', 'barbeque', 'barbers', 'barre classes', 'bars', 'bartenders', 'bartending schools', 'battery stores', 'batting cages', 'beaches', 'beauty & spas', 'bed & breakfast', 'beer', 'beer bar', 'beer tours', 'bike rentals', 'bike repair/maintenance', 'bikes', 'blow dry/out services', 'boat charters',

# Create JSON of Map Grid of Boxes

### Filter city area out of all businesses

In [9]:
# Cleveland Area
min_long = -82.354646
max_long = -81.338411
min_lat = 41.114023
max_lat = 41.730589

city_biz = business[(business.latitude < max_lat) &
                         (business.latitude > min_lat) &
                         (business.longitude < max_long) &
                         (business.longitude > min_long)]

### Compute lat and long increments

In [10]:
# Compute degrees lat per mile (N-S) and degrees long per mile (E-W)
ns_deg_per_mi = 1/69.01
ew_deg_per_mi = 1/(math.cos(
                            statistics.mean([min_lat, max_lat])*(math.pi/180)
                            ) * 69.1710411)

# Compute miles NS and EW for selected city area
miles_ns = (max_lat - min_lat)/ns_deg_per_mi
miles_ew = (max_long - min_long)/ew_deg_per_mi

delta_long = (max_long - min_long)/miles_ew
delta_lat = (max_lat - min_lat)/miles_ns

### Get counts of number of each category in a grid box

In [11]:
def get_category_counts(minimum_lat, minimum_long, d_lat, d_long):
    category_dict = defaultdict(int)
    for column, biz in business.iterrows():
        if ((biz['latitude'] > minimum_lat) and (biz['latitude'] < minimum_lat + d_lat)) and ((biz['longitude'] > minimum_long) and (biz['longitude'] < minimum_long + d_long)):
            category_dict[biz['categories']] += 1
    if not category_dict:
        category_dict['no business found in this area'] += 1
    return category_dict
        

### Create list of grid box dictionaries

In [12]:
boxes = []
 
i = min_lat
while i < max_lat:
    j = min_long
    while j < max_long:
        cat_counts = get_category_counts(i, j, delta_lat, delta_long)
        box = {'min_lat': i,
              'max_lat': i + delta_lat,
              'min_long': j,
              'max_long': j + delta_long,
              'category_counts': cat_counts}
        print(box)
        boxes.append(box)
        j += delta_long
    i += delta_lat
        

{'min_lat': 41.114023, 'max_lat': 41.12851365352848, 'min_long': -82.354646, 'max_long': -82.33536634613124, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.114023, 'max_lat': 41.12851365352848, 'min_long': -82.33536634613124, 'max_long': -82.31608669226247, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.114023, 'max_lat': 41.12851365352848, 'min_long': -82.31608669226247, 'max_long': -82.29680703839371, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.114023, 'max_lat': 41.12851365352848, 'min_long': -82.29680703839371, 'max_long': -82.27752738452494, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.114023, 'max_lat': 41.12851365352848, 'min_long': -82.27752738452494, 'max_long': -82.25824773065618, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area'

{'min_lat': 41.114023, 'max_lat': 41.12851365352848, 'min_long': -81.60273949911819, 'max_long': -81.58345984524942, 'category_counts': defaultdict(<class 'int'>, {'department stores': 1, 'drugstores': 1, 'health & medical': 1, 'public services & government': 1, 'hair salons': 1, 'fitness & instruction': 1, 'jewelry': 1, 'martial arts': 1, 'seafood': 1, 'skin care': 1, 'grocery': 1, 'ice cream & frozen yogurt': 1, 'chinese': 1})}
{'min_lat': 41.114023, 'max_lat': 41.12851365352848, 'min_long': -81.58345984524942, 'max_long': -81.56418019138066, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.114023, 'max_lat': 41.12851365352848, 'min_long': -81.56418019138066, 'max_long': -81.5449005375119, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.114023, 'max_lat': 41.12851365352848, 'min_long': -81.5449005375119, 'max_long': -81.52562088364313, 'category_counts': defaultdict(<class 'int'>,

{'min_lat': 41.12851365352848, 'max_lat': 41.143004307056955, 'min_long': -81.94977326875595, 'max_long': -81.93049361488718, 'category_counts': defaultdict(<class 'int'>, {'venues & event spaces': 1})}
{'min_lat': 41.12851365352848, 'max_lat': 41.143004307056955, 'min_long': -81.93049361488718, 'max_long': -81.91121396101842, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.12851365352848, 'max_lat': 41.143004307056955, 'min_long': -81.91121396101842, 'max_long': -81.89193430714965, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.12851365352848, 'max_lat': 41.143004307056955, 'min_long': -81.89193430714965, 'max_long': -81.87265465328089, 'category_counts': defaultdict(<class 'int'>, {'candle stores': 1, 'hotels & travel': 1, 'shopping': 1, 'race tracks': 1, 'desserts': 1, 'gyms': 1, 'heating & air conditioning/hvac': 1, 'local services': 1, 'seafood': 1})}
{'min_lat': 41.128513653

{'min_lat': 41.12851365352848, 'max_lat': 41.143004307056955, 'min_long': -81.4870615759056, 'max_long': -81.46778192203683, 'category_counts': defaultdict(<class 'int'>, {'american (new)': 1, 'massage': 1, 'bakeries': 1, 'restaurants': 6, 'tacos': 1, 'home services': 1, 'local flavor': 1, 'electricians': 1, 'antiques': 1, 'shopping': 2, 'windows installation': 1, 'japanese': 1, 'nightlife': 1, 'plumbing': 1, 'beauty & spas': 1, 'food': 1, 'bars': 2, 'lounges': 1, 'automotive': 2, 'local services': 1, 'landmarks & historical buildings': 1, 'gift shops': 1, 'djs': 1, 'american (traditional)': 1, 'auto repair': 1, 'hotels & travel': 1, 'libraries': 1, 'gyms': 1, 'burgers': 1, 'sporting goods': 1, 'arts & crafts': 1, 'ice cream & frozen yogurt': 1, 'kitchen & bath': 1})}
{'min_lat': 41.12851365352848, 'max_lat': 41.143004307056955, 'min_long': -81.46778192203683, 'max_long': -81.44850226816807, 'category_counts': defaultdict(<class 'int'>, {'pizza': 1, 'laundry services': 1, 'beauty & spa

{'min_lat': 41.143004307056955, 'max_lat': 41.15749496058543, 'min_long': -81.85337499941213, 'max_long': -81.83409534554336, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.143004307056955, 'max_lat': 41.15749496058543, 'min_long': -81.83409534554336, 'max_long': -81.8148156916746, 'category_counts': defaultdict(<class 'int'>, {'optometrists': 1, 'chiropractors': 1})}
{'min_lat': 41.143004307056955, 'max_lat': 41.15749496058543, 'min_long': -81.8148156916746, 'max_long': -81.79553603780583, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.143004307056955, 'max_lat': 41.15749496058543, 'min_long': -81.79553603780583, 'max_long': -81.77625638393707, 'category_counts': defaultdict(<class 'int'>, {'towing': 1})}
{'min_lat': 41.143004307056955, 'max_lat': 41.15749496058543, 'min_long': -81.77625638393707, 'max_long': -81.7569767300683, 'category_counts': defaultdict(<class 'int'>, {'flo

...

{'min_lat': 41.67915848761057, 'max_lat': 41.69364914113905, 'min_long': -82.00761223036224, 'max_long': -81.98833257649348, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.67915848761057, 'max_lat': 41.69364914113905, 'min_long': -81.98833257649348, 'max_long': -81.96905292262471, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.67915848761057, 'max_lat': 41.69364914113905, 'min_long': -81.96905292262471, 'max_long': -81.94977326875595, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.67915848761057, 'max_lat': 41.69364914113905, 'min_long': -81.94977326875595, 'max_long': -81.93049361488718, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.67915848761057, 'max_lat': 41.69364914113905, 'min_long': -81.93049361488718, 'max_long': -81.91121396101842, 'category_counts': defaultdict(

{'min_lat': 41.69364914113905, 'max_lat': 41.708139794667524, 'min_long': -82.29680703839371, 'max_long': -82.27752738452494, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.69364914113905, 'max_lat': 41.708139794667524, 'min_long': -82.27752738452494, 'max_long': -82.25824773065618, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.69364914113905, 'max_lat': 41.708139794667524, 'min_long': -82.25824773065618, 'max_long': -82.23896807678742, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.69364914113905, 'max_lat': 41.708139794667524, 'min_long': -82.23896807678742, 'max_long': -82.21968842291865, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.69364914113905, 'max_lat': 41.708139794667524, 'min_long': -82.21968842291865, 'max_long': -82.20040876904989, 'category_counts': default

{'min_lat': 41.69364914113905, 'max_lat': 41.708139794667524, 'min_long': -81.5449005375119, 'max_long': -81.52562088364313, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.69364914113905, 'max_lat': 41.708139794667524, 'min_long': -81.52562088364313, 'max_long': -81.50634122977436, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.69364914113905, 'max_lat': 41.708139794667524, 'min_long': -81.50634122977436, 'max_long': -81.4870615759056, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.69364914113905, 'max_lat': 41.708139794667524, 'min_long': -81.4870615759056, 'max_long': -81.46778192203683, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.69364914113905, 'max_lat': 41.708139794667524, 'min_long': -81.46778192203683, 'max_long': -81.44850226816807, 'category_counts': defaultdic

{'min_lat': 41.708139794667524, 'max_lat': 41.722630448196, 'min_long': -81.83409534554336, 'max_long': -81.8148156916746, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.708139794667524, 'max_lat': 41.722630448196, 'min_long': -81.8148156916746, 'max_long': -81.79553603780583, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.708139794667524, 'max_lat': 41.722630448196, 'min_long': -81.79553603780583, 'max_long': -81.77625638393707, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.708139794667524, 'max_lat': 41.722630448196, 'min_long': -81.77625638393707, 'max_long': -81.7569767300683, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.708139794667524, 'max_lat': 41.722630448196, 'min_long': -81.7569767300683, 'max_long': -81.73769707619954, 'category_counts': defaultdict(<class 'i

{'min_lat': 41.722630448196, 'max_lat': 41.737121101724476, 'min_long': -82.10401049970606, 'max_long': -82.0847308458373, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.722630448196, 'max_lat': 41.737121101724476, 'min_long': -82.0847308458373, 'max_long': -82.06545119196853, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.722630448196, 'max_lat': 41.737121101724476, 'min_long': -82.06545119196853, 'max_long': -82.04617153809977, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.722630448196, 'max_lat': 41.737121101724476, 'min_long': -82.04617153809977, 'max_long': -82.026891884231, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 41.722630448196, 'max_lat': 41.737121101724476, 'min_long': -82.026891884231, 'max_long': -82.00761223036224, 'category_counts': defaultdict(<class 'int

### Write to file

In [12]:
with open('az_grid_boxes_category_counts.json', 'w') as outfile:
    json.dump(boxes, outfile)